In [ ]:
! export NCS_RUN_DIR=~/nso-lab-rundir
! export REPO_DIR=~/src/nso-restconf-dns-example

In [ ]:
! source $NCS_DIR/ncsrc

In [ ]:
! make -C $REPO_DIR build

In [ ]:
from dns_restconf import DnsHandler

USERNAME = "admin"
PASSWORD = "admin"
BASE_URL = "http://localhost:8080/restconf"

restconf = DnsHandler(USERNAME, PASSWORD, BASE_URL)

In [ ]:
restconf.list_devices_in_nso()

In [ ]:
restconf.nso_sync_from_single_device(device="ex1")

In [ ]:
restconf.nso_sync_from()

In [ ]:
restconf.add_dns_server(device="ex1", dns_server="1.1.1.1")

In [ ]:
restconf.check_dns_config(device="ex1")

In [ ]:
restconf.dry_run_add_dns_server(device="ex1", dns_server="2.2.2.2")

In [ ]:
restconf.check_dns_config(device="ex1")

In [ ]:
restconf.add_dns_server(device="ex1", dns_server="2.2.2.2")

In [ ]:
restconf.check_dns_config(device="ex1")

In [ ]:
restconf.list_rollback_files()

In [ ]:
restconf.apply_rollback_file(rollback_id=0)

In [ ]:
restconf.check_dns_config(device="ex1")

In [ ]:
restconf.dry_run_add_dns_server(device="ex1", dns_server="3.3.3.3")

In [ ]:
restconf.check_dns_config(device="ex1")

In [ ]:
restconf.add_dns_server(device="ex1", dns_server="3.3.3.3")

In [ ]:
restconf.check_dns_config(device="ex1")